In [ ]:
import requests
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt

# 1. Obtener lista de países y territorios del Banco Mundial
resp = requests.get("https://api.worldbank.org/v2/country",params={"format": "json", "per_page": 400})

data = resp.json()
paises = data[1]

# Crear tabla con datos básicos de cada país (DataFrame)
df_countries = pd.DataFrame([
    {
        "name": p["name"],
        "iso3Code": p["id"],
        "region": p["region"]["value"],
        "adminregion": p["adminregion"]["value"]
    }
    for p in paises
])

# Flitro para quitar agregados ( adminregion = "" )
df_countries = df_countries[df_countries["adminregion"] != ""].reset_index(drop=True)

# *************************************
# **********  IMPORTANTE *************
# *************************************
#Comenta y descomenta según convenga para descargar el .csv

descargar = True
# descargar = False

#Usamos funcion para encaptular toda la logica de descarga
def logica_descarga(iso3, indicator_code, start_year=1980, end_year=2023):
    
    #URL para el get de un pais
    url = f"https://api.worldbank.org/v2/country/all/indicator/{indicator_code}"
    # Los parametros ponemos 20000 para coger todas las pag 
    params = {"format": "json","per_page": 20000,"date": f"{start_year}:{end_year}"}

    #Hacemos la peticion a la API
    resp = requests.get(url, params=params)
    # Respuesta = json
    data = resp.json()

    #convertimos la lista a diccionaro año:valor para acceder mas rapido
    series = {}
    for entry in data[1]:
        year = entry["date"]
        value = entry["value"]
        series[year] = value
    
    return series


if descargar:
    #Los codigos que necesitamos
    esperanza_vida = "SP.DYN.LE00.IN"
    pib = "NY.GDP.PCAP.CD"

    with open("data/datos.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerow(["country", "iso3", "region", "year", "life_expectancy", "gdp_per_capita"]) # hombre de las columnas
        
        #Recorremos cada pais del DataFrame
        for i, row in df_countries.iterrows():
            iso3 = row["iso3Code"]
            name = row["name"]
            region = row["region"]

            print(f"Descargando {name}")

            esperanza_series = logica_descarga(iso3, esperanza_vida)
            pib_series = logica_descarga(iso3, pib)

            # Recorremos cada año y hacemos una fila por año
            for year in range(1980, 2023 + 1):
                y = str(year)
                writer.writerow([
                    name,
                    iso3,
                    region,
                    y,
                    esperanza_series.get(y),
                    pib_series.get(y)
                ])

            print(f"{name} Descargado")
else:
    print("Comprueba, si sale error no la tienes descargada,cambia a descarga : True ")

    
df = pd.read_csv("data/datos.csv")
df.head()

In [ ]:
# ¿Coinciden los países más ricos con los de mayor esperanza de vida?
df_2023 = df[df["year"] == 2023]

top_ricos = df_2023.sort_values("gdp_per_capita", ascending=False).head(10)
top_vida = df_2023.sort_values("life_expectancy", ascending=False).head(10)

print("TOP 10 países más ricos en 2023:")
print(top_ricos[["country", "iso3", "gdp_per_capita"]])

print("\n")

print("TOP 10 países con mayor esperanza de vida en 2023:")
print(top_vida[["country", "iso3", "life_expectancy"]])

print("\n")

coinciden = set(top_ricos["country"]) & set(top_vida["country"])
porcentaje = (len(coinciden)/ len(df_2023)*100)

print(f"Los pasises mas rico con mayor esperanza de vida son {coinciden} Esto representa el {porcentaje:.2f}% detodos los paises analizados.")

print("\n")



In [ ]:
#¿Qué países se comportan de forma atípica respecto a la relación riqueza–bienestar?
# Mediana como referencia
mediana_pib = df_2023["gdp_per_capita"].median()
mediana_vida = df_2023["life_expectancy"].median()

# VALORES ATOPICOS NEGATIVOS
val_ato_neg = df_2023[
    (df_2023["gdp_per_capita"] > mediana_pib) &
    (df_2023["life_expectancy"] < mediana_vida)
][["country", "iso3", "gdp_per_capita", "life_expectancy"]].sort_values("gdp_per_capita", ascending=False)

# VALORES ATOPICOS POSITIVOS
val_ato_pos = df_2023[
    (df_2023["gdp_per_capita"] < mediana_pib) &
    (df_2023["life_expectancy"] > mediana_vida)
][["country", "iso3", "gdp_per_capita", "life_expectancy"]].sort_values("life_expectancy", ascending=False)

print("Los valores atopicos de ricos pero con baja esperanza de vida:")
print(val_ato_neg)

print("\n")

print("Los valores atopicos de pobres pero con alta esperanza de vida:")
print(val_ato_pos)


In [ ]:

plt.scatter(data=df_2023, x="gdp_per_capita", y="life_expectancy",
            color="lightgray", label="Resto de países")

for i, row in top_ricos.iterrows():
    plt.text(row["gdp_per_capita"], row["life_expectancy"], row["country"], fontsize=7, color = "red")

for i, row in top_vida.iterrows():
    plt.text(row["gdp_per_capita"], row["life_expectancy"], row["country"], fontsize=7,color = "green")

for i, row in val_ato_neg.iterrows():
    plt.text(row["gdp_per_capita"], row["life_expectancy"], row["country"], fontsize=7, color = "darkred")
    
for i, row in val_ato_pos.iterrows():
    plt.text(row["gdp_per_capita"], row["life_expectancy"], row["country"], fontsize=7, color = "blue")
    
plt.xlabel("PIB")
plt.ylabel("Años de esperanza de vida")

print("Como leyenda podemos ver que los paises:")
print("\n")
print("ROJOS son los mas ricos")
print("VERDES son los mas esperanza de vida")
print("ROJO OSCURO son los valor atopico mas negativo mas altos")
print("AZUL son los valor atopico mas negativo mas bajos")
print("\n")
print( "En conclusión, podemos sacar que, a grandes rasgos que el PIB y esperanza de vida relacionada pero no es perfecta, ya que hay paises ricos con esperanza de vida bajo y viceversa" )